In [1]:
# import os
# os.getcwd()
#!pip install dotmap

In [1]:
import sys

# make sure to use position 1
sys.path.insert(1, "/jet/prs/workspace/Keras-Project-Template-Jupyter")

In [2]:
from utils.config import process_config
from utils.dirs import create_dirs
from utils.utils import get_args

from data_loader.prostate_dist_dvh_data_loader import ProstateDistDvhDataLoader
from models.transfer_3block_vgg16_model import Transfer3BlockVGG16Model
from models.transfer_4block_vgg16_model import Transfer4BlockVGG16Model

from trainers.simple_trainer import SimpleTrainer

import h5py
import numpy as np
from sklearn.model_selection import StratifiedKFold


Using TensorFlow backend.


In [3]:
def main(config_file):
    # capture the config path from the run arguments
    # then process the json configuration file
    try:
        config = process_config(config_file)
        #print config
    except:
        assert False, "missing or invalid arguments"
        
    # create the experiments dirs
    print('Create the experiments dirs.')
    create_dirs([config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

    print('Create the data generator.')
    data_loader = ProstateDistDvhDataLoader(config)
    
    
    # cross validation
    cvscores = []
    kfold = StratifiedKFold(
        n_splits=config.cross_validation.n_splits,
        shuffle=config.cross_validation.shuffle,
        random_state=config.cross_validation.random_state
    )
    
    for train, test in kfold.split(data_loader.get_data()[0], data_loader.get_data()[1]):
        
        # print('Create the model.')
        model = Transfer4BlockVGG16Model(config)
        #Transfer4BlockRnnVGG16Model(config)
        
        # train or train-val 
        # print('Create the trainer')
        trainer = SimpleTrainer(model.model, data_loader.get_data(), config)

        print('Start training/validating the model.')
        # trainer.train()
        scores = trainer.train_val(train, test)
    
        for metric, score in zip(model.model.metrics_names, scores):
            print("%s: %.2f%%" % (metric, score*100))

        cvscores.append(scores)

    means = np.mean(cvscores, axis=0)
    stds = np.std(cvscores, axis=0)

    for metric, mean, std in zip(model.model.metrics_names, means, stds):
        print("%s: %.2f%% (+/- %.2f%%)" % (metric, mean * 100, std * 100))

if __name__ == '__main__':
    main("configs/vgg16_4blocks_from_config.json")

Create the experiments dirs.
Create the data generator.


IOError: Unable to open file (unable to open file: name = '/home/cci_hit_uncc/maryam/data/case_216/datapack_rectum_distance3d_2018.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#!pip install dotmap
# !pip install comet_ml
#!pip install --upgrade tensorflow tensorflow-tensorboard
#!pip install comet_ml
# !pip install tensorflow==1.8.0

In [ ]:
#!tensorboard --logdir="/Users/mtavako1/Documents/Research/__Radiation_Therapy/Code/1_KerasLearning/Keras-Project-Template-Jupyter/experiments/2018-10-29/vgg16_3blocks_from_config/"

In [ ]:
# get_config_from_json("configs/simple_mnist_config.json")

In [ ]:
# https://github.com/keunwoochoi/keras_callbacks_example
# GAN: https://www.youtube.com/watch?v=0VPQHbMvGzg

In [ ]:
tuple([1]) + (3,4)
